In [19]:
import numpy as np

from MuyGPyS._test.sampler import UnivariateSampler, print_results
from MuyGPyS.gp import MuyGPS
from MuyGPyS.gp.deformation import Isotropy, l2
from MuyGPyS.gp.hyperparameter import AnalyticScale, Parameter
from MuyGPyS.gp.kernels import Matern
from MuyGPyS.gp.noise import HomoscedasticNoise
from MuyGPyS.neighbors import NN_Wrapper
from MuyGPyS.optimize import Bayes_optimize
from MuyGPyS.optimize.batch import sample_batch
from MuyGPyS.optimize.loss import lool_fn

In [20]:
np.random.seed(0)


In [21]:
data_count = 3000
train_ratio = 0.075

nugget_noise = HomoscedasticNoise(1e-14)
measurement_noise = HomoscedasticNoise(1e-7)

sim_length_scale = Parameter(0.05)
sim_smoothness = Parameter(2.0)

In [22]:
sampler = UnivariateSampler(
    data_count=data_count,
    train_ratio=train_ratio,
    kernel=Matern(
        smoothness=sim_smoothness,
        deformation=Isotropy(
            l2,
            length_scale=sim_length_scale,
        ),
    ),
    noise=nugget_noise,
    measurement_noise=measurement_noise,
)

# features are "x values", responses are "f(x)"
train_features, test_features = sampler.features()
train_responses, test_responses = sampler.sample()


In [23]:
data = np.stack((train_features, train_responses), axis=1)
print(data.shape)

(225, 2)


Everything above is simply to get data for `train_features`, `test_features`, `train_responses`, and `test_responses` 

In [24]:
def create_gp(length, noise):
    # Initialize the type of GP to use
    # Here we assume noise level and length_scale are known, and optimize for smoothness
    gp = MuyGPS(
        kernel=Matern(
            smoothness=Parameter("log_sample", (0.1, 5.0)),
            deformation=Isotropy(
                l2,
                length_scale=sim_length_scale,
            ),
        ),
        noise=measurement_noise,
        scale=AnalyticScale(),
    )

    return gp

def train_gp(gp, batch_indices, batch_nn_indices, data):
    (
        batch_crosswise_dists,
        batch_pairwise_dists,
        batch_targets,
        batch_nn_targets,
    ) = gp.make_train_tensors(
        batch_indices,
        batch_nn_indices,
        data[:,0],
        data[:,1],
    )

    # # Compute covariances for each of the pairwise distances
    # Kcross = gp.kernel(batch_crosswise_dists)
    # Kin = gp.kernel(batch_pairwise_dists)

    # Solve/optimize the GP for best hyperparameters
    # We can use scipy.lbfgs here as an alternative
    trained_gp = Bayes_optimize(
        gp,
        batch_targets,
        batch_nn_targets,
        batch_crosswise_dists,
        batch_pairwise_dists,
        loss_fn=lool_fn,
        verbose=True,
        random_state=1,
        init_points=5,
        n_iter=15,
    )

    # Optimize scale parameter separately
    trained_gp = trained_gp.optimize_scale(batch_pairwise_dists, batch_nn_targets)

    return trained_gp

In [25]:
# Set up batch of data for training
nn_count = 30
batch_count = 500

nbrs_lookup = NN_Wrapper(train_features, nn_count, nn_method="exact", algorithm="ball_tree")
batch_indices, batch_nn_indices = sample_batch(
    nbrs_lookup, batch_count, sampler.train_count
)

gp = create_gp(sim_length_scale, measurement_noise)
gp = train_gp(gp, batch_indices, batch_nn_indices, data)




parameters to be optimized: ['smoothness']
bounds: [[0.1 5. ]]
initial x0: [0.92898658]
|   iter    |  target   | smooth... |
-------------------------------------
| 1         | 1826.4356 | 0.9289865 |
| 2         | 2359.0765 | 2.1434078 |
| 3         | 1952.6510 | 3.6295900 |
| 4         | 614.38904 | 0.1005604 |
| 5         | 2308.5389 | 1.5814296 |
| 6         | 1707.0528 | 0.8191038 |
| 7         | 1480.1007 | 5.0       |
| 8         | 2201.4775 | 2.8303648 |
| 9         | 2373.3739 | 1.8833036 |
| 10        | 2373.2662 | 1.9964344 |
| 11        | 2374.7103 | 1.9380261 |
| 12        | 2374.7103 | 1.9379595 |
| 13        | 2374.7103 | 1.9378658 |
| 14        | 2374.7103 | 1.9377919 |
| 15        | 2374.7104 | 1.9377218 |
| 16        | 2374.7104 | 1.9376629 |
| 17        | 2374.7103 | 1.9374795 |
| 18        | 2374.6874 | 1.9449324 |
| 19        | 2374.6564 | 1.9265826 |
| 20        | 2374.6475 | 1.9497154 |
| 21        | 2374.6505 | 1.9259924 |


In [26]:
def predict(gp, test_features, train_data):
    # Get locations for inference
    test_count = test_features.shape[0]
    indices = np.arange(test_count)
    test_nn_indices, _ = nbrs_lookup.get_nns(test_features)

    # Get distance tensors
    (
        test_crosswise_dists,
        test_pairwise_dists,
        test_nn_targets,
    ) = gp.make_predict_tensors(
        indices,
        test_nn_indices,
        test_features,
        train_data[:,0],
        train_data[:,1],
    )

    # Get covariance tensors
    Kcross = gp.kernel(test_crosswise_dists)
    Kin = gp.kernel(test_pairwise_dists)

    # Make prediction
    predictions = gp.posterior_mean(Kin, Kcross, test_nn_targets)

    return predictions

In [27]:
predictions = predict(gp, test_features, data)

In [28]:
# print results of prediction
variances = gp.posterior_variance(Kin, Kcross)
confidence_intervals = np.sqrt(variances) * 1.96
coverage = np.count_nonzero(np.abs(test_responses - predictions) < confidence_intervals) / test_count
print_results(
    test_responses, ("optimized", gp, predictions, variances, confidence_intervals, coverage)
)

# plot prediction and confidence intervals
sampler.plot_results(("optimized", predictions, confidence_intervals))

NameError: name 'Kin' is not defined